In [1]:
import pandas as pd
import numpy as np
from funcs import *

In [2]:
data = downloadData()
sigma = historicalVol()
S0 = intitialPrice(data)
r = selic() # as vezes precisa rodar duas vezes pra funcionar...?

df = pd.read_csv("data/petr4_data.csv")
K = np.mean(df["Strike"])
call = blackScholesCall(S0, K, sigma, r, 0.5) # 0.5 ano --> 6 meses

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Volatilidade histórica anual: 25.11%


In [6]:
print(f"Valor de call: ${call[0]:.2f}")

Valor de call: $34.54
